In [5]:
# Last updated: Sep/26/2024
# This file is used to sample the event


# the logic is following:
# start_run and end_run will determine the range of runs for search
# days_apart is used to evenly space the sample day
# within a day, d_date is used to search individual data point
# in order to avoid extreme value within short tiem, 
# thus at the time need to be sampled, the data within sampled time +- t_tolerance/2 will be sampled
# and the final data set will be only choose 1/8 of all set in such data to reduce computation time in analyzer


# note: this file is only used to select run and event, the output is run and event number, 
# will not directly print out data number



import numpy as np
import matplotlib.pyplot as plt
import sys
import os
import inspect
import warnings
import time
from datetime import datetime
from datetime import timedelta
# Start the timer
start_time = time.time()
import random
warnings.simplefilter(action='ignore', category=FutureWarning)

sys.path.append('home/youwei/project_beacon/2024_10')
from Reader_class import Reader

import ROOT

def loadTriggerTypes(reader):
    try:
        N = reader.head_tree.Draw("trigger_type", "", "goff") 
        trigger_type = np.frombuffer(reader.head_tree.GetV1(), np.dtype('float64'), N).astype(int)
    except Exception as e:
        print(f'\nError in {inspect.stack()[0][3]}')
        print('Error while trying to copy header elements to attrs.')
        print(e)
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print(exc_type, fname, exc_tb.tb_lineno)
        return e
    return trigger_type

def loadreadout_t(reader):
    try:
        N = reader.head_tree.Draw("readout_time", "", "goff") 
        readout_t = np.frombuffer(reader.head_tree.GetV1(), np.dtype('float64'), N).astype(float)
    except Exception as e:
        print(f'\nError in {inspect.stack()[0][3]}')
        print('Error while trying to copy header elements to attrs.')
        print(e)
        exc_type, exc_obj, exc_tb = sys.exc_info()
        fname = os.path.split(exc_tb.tb_frame.f_code.co_filename)[1]
        print(exc_type, fname, exc_tb.tb_lineno)
        return e
    return readout_t

# Main parameters
datapath = "/project/avieregg/beacon-flower8/root/"
seed = 42
random.seed(seed)

start_run = 700
end_run = 1500
t_tolerance = 120
d_date=1./48. # how many days apart for each set of data we are looking for, used for one day plot
days_apart= 60 # after taking one day of data, how many date apart do we want to take another data 
#maximum_size=96


#generate an array for the date we what to collect data for:
# Generate an array of filename
start_date = datetime(2024, 2, 14, 0, 0, 1)
end_date = datetime(2024, 5, 23, 0, 0, 1)
data_days_arr=[]
test_date=start_date
output_filename=[]
while test_date<end_date:
    data_days_arr.append(test_date)
    date_str = test_date.strftime("%Y-%m-%d")
    output_filename.append("sampling_at_"+date_str)
    test_date = test_date + timedelta(days=days_apart)



print(output_filename)

['sampling_at_2024-02-14', 'sampling_at_2024-04-14']


In [ ]:
for j in range(len(data_days_arr)):
    

    run_array = []
    readout_t_array = []
    entry_array = []
    evernt_num=[]
    
    dt = d_date * 24 * 3600
    start_t = data_days_arr[j].timestamp()
    
    #last_data_reader = Reader(datapath, end_run)
    end_t = start_t + 24*3600
    
    # Generate the array of timestamps with intervals of dt
    time_to_find_array = np.arange(start_t, end_t + 1, dt)
    
    for run in np.arange(start_run, end_run + 1):
        time_arr=[]
        reader = Reader(datapath, run)
        readout_time = loadreadout_t(reader)
    
        time_in_range = [readout_time[0]-15 <= time <= readout_time[-1]+15 for time in time_to_find_array]
    
        if not any(time_in_range):
            continue
        
        for i in range(len(time_in_range)):
            if time_in_range[i]:
                time_arr.append(time_to_find_array[i])
                
        trigger_types = loadTriggerTypes(reader)
        
        for time_to_find in time_arr:
            delta_t = np.abs(readout_time - time_to_find)
            eventids = np.where((trigger_types == 1) & (delta_t < t_tolerance))[0]
            
            if len(eventids) > 0:
                random_id=[]
                random_id = np.random.choice(eventids, size=(len(eventids)//8), replace=False)
            else:
                print("Error, No event IDs found.")
                random_id = None
                break
    
            for ids in random_id:
            #for ids in eventids:
                run_array.append(run)
                readout_t_array.append(readout_time[ids])
                entry_array.append(ids)
    
        
        # if maximum_size != 0 and len(run_array)>=maximum_size:
        #     break
    
    # Convert readout times to datetime objects
    datetime_array = [datetime.fromtimestamp(rt) for rt in readout_t_array]
    
    print(run_array)
    print(datetime_array)
    print(entry_array)
    
    # Get the current date and time
    generation_date = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    
    
    with open(output_filename[j], "w") as f:
        f.write(f"Data is selected from run {start_run} to run {end_run}, starting with a reference time {reference_t} "
                f"+- {t_tolerance}s with a separation date of {d_date} days\n")
        f.write(f"Data Generated On: {generation_date}\n\n")
        f.write("Run Array:\n")
        f.write(", ".join(map(str, run_array)) + "\n\n")
        f.write("Readout Time Array (timestamps):\n")
        f.write(", ".join(map(str, readout_t_array)) + "\n\n")
        f.write("Entry Array:\n")
        f.write(", ".join(map(str, entry_array)) + "\n\n")
        f.write("Readout Time Array (dates):\n")
        f.write(", ".join(map(str, datetime_array)) + "\n\n")
    
    # Stop the timer
    end_time = time.time()
    
    # Calculate the execution time
    execution_time = end_time - start_time
    print(f"{len(run_array)}entry ploted")
    print(f"Data has been written to {output_filename}")
    print(f"Execution Time: {execution_time:.2f} seconds")